In [ ]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
def GetCNN():
  model = models.Sequential()
  
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(20, 20, 1)))
  model.add(layers.BatchNormalization())
  
  model.add(layers.Conv2D(32, (3, 3), activation='relu'))
  model.add(layers.BatchNormalization())

  model.add(layers.Conv2D(32, (3, 3), activation='relu'))
  model.add(layers.BatchNormalization())

  model.add(layers.Dropout(0.1))
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))# fully connected
  model.add(layers.Dropout(0.1))
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.1))
  
  model.add(layers.Dense(10,activation='softmax', use_bias=True))
  return model 
def getModel(modelName="ResNet50", input_shape=(32,32,1)):

  input_layr = tf.keras.Input(shape=input_shape)


  # Xception, VGG16, VGG19,ResNet101,ResNet152, InceptionV3, InceptionResNetV2,MobileNet,MobileNetV2,DenseNet121,DenseNet169,DenseNet201,EfficientNetB0,EfficientNetB1.. B7
  
  model=None 
  if modelName=="CNN":
    model = GetCNN()
    return model

## Dataset Link : https://github.com/adhishthite/sound-mnist 

In [ ]:
!git clone https://github.com/adhishthite/sound-mnist.git

In [ ]:
cd sound-mnist/

/content/sound-mnist


In [ ]:
from utils import model, wav2mfcc, get_data
import test
import utils

In [ ]:
import keras
keras_callback = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=1,
                                             write_graph=True, write_images=True)
from sklearn.model_selection import train_test_split
import numpy as np
import librosa
import os
from tensorflow.keras.utils import to_categorical
def wav2mfcc(file_path, max_pad_len=20):
    wave, sr = librosa.load(file_path, mono=True, sr=None)
    wave = wave[::3]
    mfcc = librosa.feature.mfcc(wave, sr=8000)
    pad_width = max_pad_len - mfcc.shape[1]
    mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    return mfcc

def get_data():

    labels = []
    mfccs = []

    for f in os.listdir('./recordings'):
        if f.endswith('.wav'):
            # MFCC
            mfccs.append(wav2mfcc('./recordings/' + f))

            # List of labels
            label = f.split('_')[0]
            labels.append(label)

    return np.asarray(mfccs), to_categorical(labels)

# if __name__ == '__main__':
#     mfccs, labels = get_data()
#     print(mfccs.shape)
#     print(labels.shape)

mfccs, labels = get_data()

dim_1 = mfccs.shape[1]
dim_2 = mfccs.shape[2]
channels = 1
classes = 10

X = mfccs
X = X.reshape((mfccs.shape[0], dim_1, dim_2, channels))
y = labels

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)


X_train.shape


/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=781
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=984
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=785
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=718
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1300
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/core/spectrum.py:224: UserWarning: n_fft=2048 is too small for input signal of length=1656
  n_fft, y.shape[-1]
/usr/local/lib/python3.7/dist-packages/librosa/cor

(1350, 20, 20, 1)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
cnn_model = getModel(modelName="CNN", input_shape=(20,20,1))
print(cnn_model.summary())
cnn_model.compile(loss  = tf.keras.losses.CategoricalCrossentropy(),
              metrics = ["accuracy"],
              optimizer = tf.keras.optimizers.Adam(learning_rate=0.01))
callbacks = ModelCheckpoint("BestFSDD_.h5", monitor='val_accuracy',
                                    mode='max',
                                    save_best_only=True,
                                    verbose=1)

cnn_model.fit(X_train, y_train, batch_size=64, epochs=50, verbose=1, validation_split=0.1, callbacks=[callbacks])

cnn_model = keras.models.load_model("BestFSDD_.h5")
print("Test Accuracy is ", cnn_model.evaluate(X_test,y_test)[1])

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 18, 18, 32)        320       
_________________________________________________________________
batch_normalization_24 (Batc (None, 18, 18, 32)        128       
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 16, 16, 32)        9248      
_________________________________________________________________
batch_normalization_25 (Batc (None, 16, 16, 32)        128       
_________________________________________________________________
conv2d_26 (Conv2D)           (None, 14, 14, 32)        9248      
_________________________________________________________________
batch_normalization_26 (Batc (None, 14, 14, 32)        128       
_________________________________________________________________
dropout_24 (Dropout)         (None, 14, 14, 32)       